In [1]:
pip install pdfplumber python-docx pytesseract opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 27.9 MB/s eta 0:00:00


In [8]:
import torch
import pdfplumber
import cv2
import pytesseract

In [3]:
from transformers import T5Tokenizer, T5ForConditionalGeneration,BartForConditionalGeneration

In [4]:
from docx import Document
from tkinter import Tk,filedialog

In [6]:
"""extract from pdf"""
def extractfrompdf(pdf_path):
  text=""
  text = ""
  with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"
  return text.strip()

In [7]:
"""Extract text from a DOCX file"""
def extract_text_from_docx(docx_path):
  doc = Document(docx_path)
  return "\n".join([para.text for para in doc.paragraphs])


In [9]:
"""Extract text from an image using Tesseract OCR."""
def extract_text_from_image(image_path):
  image = cv2.imread(image_path)
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  text = pytesseract.image_to_string(gray)
  return text.strip()

In [1]:
def summarize_text(text,model_name="t5"):
  if not text:
    return "No text found in the document"
  if model_name=="t5":
    model = T5ForConditionalGeneration.from_pretrained("t5-small")
    tokenizer = T5Tokenizer.from_pretrained("t5-small")
    input_text = "summarize: " + text
  elif model_name=="bart":
    model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
    tokenizer = T5Tokenizer.from_pretrained("t5-small")
    input_text = "summarize: " + text
  else:
      raise ValueError("Invalid model name. Choose 't5' or 'bart'.")
  inputs = tokenizer.encode(input_text, return_tensors="pt", max_length=1024, truncation=True)
  summary_ids = model.generate(inputs, max_length=200, min_length=50, length_penalty=2.0, num_beams=4, early_stopping=True)

  return tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [8]:
"""Open a file dialog to select a document."""
#from tkinter import Tk, filedialog
def select_file():
  """Prompts the user to enter the file path manually."""
  file_path = input("Enter the path to your document: ")
  return file_path

In [ ]:
if __name__ == "__main__":
    file_path = select_file()

    if not file_path:
        print("No file selected.")
        exit()

    # Determine file type and extract text
    if file_path.endswith(".pdf"):
        text = extract_text_from_pdf(file_path)
    elif file_path.endswith(".docx"):
        text = extract_text_from_docx(file_path)
    elif file_path.endswith((".jpg", ".png")):
        text = extract_text_from_image(file_path)
    else:
        print("Unsupported file format!")
        exit()

    print("\nExtracted Text:\n", text[:1000])  # Display the first 1000 characters

    # Summarize the text
    model_choice = input("\nChoose summarization model (t5/bart): ").strip().lower()
    summary = summarize_text(text, model_choice)

    print("\nSummary:\n", summary)

In [10]:
from google.colab import files

uploaded = files.upload()  # Opens a file selection dialog


Saving INDEPENDENT CONTRACTOR AGREEMENT.pdf to INDEPENDENT CONTRACTOR AGREEMENT.pdf
